# Set up your environment

Let's use a `.env` file and the `dotenv` library. This allows you to keep your API key secure and separated from your code.

Here’s how you can setup to load the OpenAI API key using a `.env` file:

### Step 1: Install Dependencies
Make sure you have `python-dotenv` installed if you're using `.env` files:
```bash
pip install python-dotenv
pip install -U langsmith
pip install openai
pip install langchain_openai
```

### Step 2: Create a `.env` File
In your project directory, create a `.env` file with the following content:
```
OPENAI_API_KEY=your-api-key-here
```

### Step 3: Test

In [1]:
import os
import openai
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Function to load environment variables or raise an error if not found
def get_env_var(var: str):
    value = os.getenv(var)
    if value is None:
        raise ValueError(f"{var} not found in environment variables. Make sure it is set in your .env file.")
    return value

# Load API keys from the environment
langchain_api_key = get_env_var("LANGCHAIN_API_KEY")
langchain_tracing_v2 = get_env_var("LANGCHAIN_TRACING_V2")
openai_api_key = get_env_var("OPENAI_API_KEY")
tavily_api_key = get_env_var("TAVILY_API_KEY")

# Now, you can use the keys in our setup as needed
from langchain_openai import ChatOpenAI
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=openai_api_key)
gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, openai_api_key=openai_api_key)

In [2]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Lance")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
gpt4o_chat.invoke(messages)

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_e375328146', 'finish_reason': 'stop', 'logprobs': None}, id='run-2ad8fa8d-e3fa-4fd3-a350-c1769f287626-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20})

In [3]:
gpt4o_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_e375328146', 'finish_reason': 'stop', 'logprobs': None}, id='run-d7c275df-b7f4-4f8a-8f55-2531bce4c0d7-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18})

In [4]:
gpt35_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-017852c1-5961-4e62-9d6b-7fb0ed53d386-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18})

In [5]:
# Set up Tavily using the API key from the environment
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_search = TavilySearchResults(max_results=3, api_key=tavily_api_key)

In [6]:
# Perform a search with Tavily
search_docs = tavily_search.invoke("What is LangGraph?")
search_docs

[{'url': 'https://www.datacamp.com/tutorial/langgraph-tutorial',
  'content': 'LangGraph is a library within the LangChain ecosystem designed to tackle these challenges head-on. LangGraph provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured manner. It simplifies the development process by enabling the creation of cyclical graphs, which are essential for developing ...'},
 {'url': 'https://towardsdatascience.com/from-basics-to-advanced-exploring-langgraph-e8c1cf4db787',
  'content': "LangGraph is a low-level framework that offers extensive customisation options, allowing you to build precisely what you need. Since LangGraph is built on top of LangChain, it's seamlessly integrated into its ecosystem, making it easy to leverage existing tools and components. However, there are areas where LangGrpah could be improved:"},
 {'url': 'https://www.youtube.com/watch?v=nmDFSVRnr4Q',
  'content': 'LangGraph is an extension of LangChain enab

In [7]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback

# Load environment variables from .env file
load_dotenv()

# Function to load environment variables or raise an error if not found
def get_env_var(var: str):
    value = os.getenv(var)
    if value is None:
        raise ValueError(f"{var} not found in environment variables. Make sure it is set in your .env file.")
    return value

# Load API keys from the environment
openai_api_key = get_env_var("OPENAI_API_KEY")

# Initialize the OpenAI LLM via LangChain
gpt_chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)

# Define a simple prompt template for conversation
prompt_template = PromptTemplate(
    input_variables=["user_input"],
    template="You are a helpful assistant. User asks: {user_input}. How would you respond?"
)

# Instead of using RunnableSequence, directly chain the prompt and model using the | operator
conversation_chain = prompt_template | gpt_chat

try:
    # Start callback for tracing OpenAI usage
    with get_openai_callback() as cb:
        print("LangChain tracing started successfully.")

        # Example conversation - LangChain will trace this operation
        user_input = "What are the benefits of using transformers in natural language processing?"
        response = conversation_chain.invoke({"user_input": user_input})

        print("Agent's Response:")
        print(response)

        # You can extend this conversation further
        follow_up = "Can you explain the difference between GPT and BERT?"
        follow_up_response = conversation_chain.invoke({"user_input": follow_up})

        print("Follow-up Response:")
        print(follow_up_response)

        # Output callback information
        print(f"\nTotal Tokens Used: {cb.total_tokens}")
        print(f"Total Cost (USD): ${cb.total_cost}")

    print("LangChain tracing ended successfully.")

except Exception as e:
    print(f"Error with LangChain tracing: {e}")


LangChain tracing started successfully.
Agent's Response:
content='Using transformers in natural language processing has several benefits, including:\n\n1. Improved performance: Transformers have shown to outperform traditional models in various NLP tasks, such as language translation, sentiment analysis, and text generation.\n\n2. Better handling of long-range dependencies: Transformers are able to capture relationships between words that are far apart in a sentence, making them more effective at understanding context and meaning.\n\n3. Scalability: Transformers can be easily scaled up to handle large amounts of data, making them suitable for processing vast amounts of text in real-world applications.\n\n4. Transfer learning: Transformers can be fine-tuned on specific tasks with relatively small amounts of data, allowing for efficient transfer learning and adaptation to new domains.\n\nOverall, using transformers in NLP can lead to more accurate and efficient language processing tasks